# NLP Review 

- Textuel Data (Text mining )
- it very large field in machine learning 
- NLTK book (Seprate python Library )
- Foundations of Statistical Natural Language Processing ( by Manning)

# NLP used For ?
- Clustering New Articles 
- Seggestion similar books 
- Analysing Comnsumer FeedBack 
- NLP process :
    - Corpus - Bag of word 
    - Featureise the words to numerics 
    - Compare feature of documents 
         - to compare this we gonna use TF- IDF we can also take a count of DR (AFC)
         
         


# Example 

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("nlp").getOrCreate()

# lets talk about tokenized and tokenization 

- Tokenization is a preocess of taking a text such as a sentence and then breaking it into list of individual terms
- for most powerfull processing textual donne see (Regular expression)

In [22]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col,udf
# col allow us to call a columns and then use the fonction using lamba expression
from pyspark.sql.types import IntegerType

In [44]:
#Create A data Frame
send_df = spark.createDataFrame([
    (0, "hi i heard about Spark"),
    (1,"I wish java could use case classes "),
    (2,"Logistic,regression,models,are,neat")
],["id","Sentences"])

In [63]:
send_df.show()

+---+--------------------+
| id|           Sentences|
+---+--------------------+
|  0|hi i heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



# Processing 

In [64]:
token = Tokenizer(inputCol="Sentences",outputCol="words")

In [73]:
regex = RegexTokenizer(inputCol="Sentences",outputCol="words", pattern = "\\W")

In [74]:
count_token = udf(lambda words:len(words), IntegerType())

In [75]:
tokenized = token.transform(send_df)
tokenized.show()

+---+--------------------+--------------------+
| id|           Sentences|               words|
+---+--------------------+--------------------+
|  0|hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [76]:
tokenized.withColumn("tokens",count_token(col("words"))).show()

+---+--------------------+--------------------+------+
| id|           Sentences|               words|tokens|
+---+--------------------+--------------------+------+
|  0|hi i heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [77]:
rg =regex.transform(send_df)
rg.show()

+---+--------------------+--------------------+
| id|           Sentences|               words|
+---+--------------------+--------------------+
|  0|hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic, regres...|
+---+--------------------+--------------------+



In [78]:
rg.withColumn("tokens",count_token(col("words"))).show()

+---+--------------------+--------------------+------+
| id|           Sentences|               words|tokens|
+---+--------------------+--------------------+------+
|  0|hi i heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [79]:
from pyspark.ml.feature import StopWordsRemover

In [82]:
sentence_dataframe = spark.createDataFrame([
    (0,["i","saw","the","green","horse"]),
    (1,["mary","had","a","little","lamb"]),
    
], ["id","tokens"])

In [83]:
remover = StopWordsRemover(inputCol="tokens",outputCol="filtred")

In [86]:
remover.transform(sentence_dataframe).show()

+---+--------------------+--------------------+
| id|              tokens|             filtred|
+---+--------------------+--------------------+
|  0|[i, saw, the, gre...| [saw, green, horse]|
|  1|[mary, had, a, li...|[mary, little, lamb]|
+---+--------------------+--------------------+



# n-gram
it s a sequence of n tokens for some integer N and the angram class can be used to transform input features to ngrams

In [87]:
from pyspark.ml.feature import NGram

In [96]:
#Create A data Frame
send_df = spark.createDataFrame([
    (0,[ "hi", "i" ,"heard", "about", "Spark"]),
    (1,["I", "wish", "java", "could", "use", "case", "classes"]),
    (2,["Logistic","regression","models","are","neat"])],["id","words"])

In [103]:
ngram = NGram(n = 3 , inputCol="words", outputCol="grams")

In [104]:
ngram.transform(send_df).select("grams").show(truncate= False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[hi i heard, i heard about, heard about Spark]                                  |
|[I wish java, wish java could, java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



this kind of ngram is usefull when we want the relationships between two words, which words alwas appear next each other 

# Term-fequency and Doc Fequency

In [105]:
from pyspark.ml.feature import HashingTF, IDF,Tokenizer
# Hashing TF for term Frequency , IDF for univer Document frequency, 

In [106]:
#Create A data Frame
send_df = spark.createDataFrame([
    (0, "hi i heard about Spark"),
    (1,"I wish java could use case classes "),
    (2,"Logistic,regression,models,are,neat")
],["id","Sentences"])

In [107]:
send_df.show()

+---+--------------------+
| id|           Sentences|
+---+--------------------+
|  0|hi i heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [108]:
token = Tokenizer(inputCol="Sentences",outputCol="words")
#regex = RegexTokenizer(inputCol="Sentences",outputCol="words", pattern = "\\W")

In [109]:
words_data = token.transform(send_df)

In [110]:
words_data.show()

+---+--------------------+--------------------+
| id|           Sentences|               words|
+---+--------------------+--------------------+
|  0|hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [112]:
hashing_td = HashingTF(inputCol="words",outputCol="rawFeatures")

In [114]:
featurized_data = hashing_td.transform(words_data)

In [115]:
idf = IDF(inputCol="rawFeatures",outputCol="features")

In [116]:
idf_model = idf.fit(featurized_data)

In [117]:
rescaled_data = idf_model.transform(featurized_data)

In [122]:
rescaled_data.select("id","features").show()
# transformed the the textuel data to numeric 

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|(262144,[18700,19...|
|  1|(262144,[19036,20...|
|  2|(262144,[11534],[...|
+---+--------------------+



In [123]:
# Count Vectorizer
from pyspark.ml.feature import CountVectorizer

In [125]:
df = spark.createDataFrame([
    
    (0,"a b c ".split(" ")),
      (0,"a b b c a ".split(" "))], [ "id","words"])

In [126]:
df.show()

+---+-----------------+
| id|            words|
+---+-----------------+
|  0|      [a, b, c, ]|
|  0|[a, b, b, c, a, ]|
+---+-----------------+



In [127]:
# take the best words and or the most pertinent work that are accored in more than one doc 
cv = CountVectorizer(inputCol="words",outputCol="features", vocabSize= 3 )

In [128]:
model = cv.fit(df)

In [132]:
result = model.transform(df).show(truncate = False)

+---+-----------------+-------------------------+
|id |words            |features                 |
+---+-----------------+-------------------------+
|0  |[a, b, c, ]      |(3,[0,1,2],[1.0,1.0,1.0])|
|0  |[a, b, b, c, a, ]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+-----------------+-------------------------+

